In [1]:
import os
import dask.dataframe as df
from operator import add
from dask.distributed import Client
import matplotlib.pyplot as plt
from dask.cache import Cache

head, tail = os.path.split(os.getcwd())
data_dir = os.path.join(head, 'data')
data_raw_dir = os.path.join(data_dir, 'raw')

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:63090")
client

Client Scheduler: tcp://127.0.0.1:63090 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [3]:
park_spot_data = df.read_csv(os.path.join(data_raw_dir, 'Places.csv'), 
                             encoding='cp1252', blocksize='512KiB')[['sNoPlace','nLongitude','nLatitude']]

list_of_borough =['Plateau-Mont-Royal',
                  'Saint-Léonard',
                  'Rosemont - La Petite-Patrie',
                  'Outremont',
                  'Ville-Marie',
                  'Mercier - Hochelaga-Maisonneuve',
                  'LaSalle',
                  'Villeray - Saint-Michel - Parc-Extension',
                  'Rivière-des-Prairies - Pointe-aux-Trembles',
                  'Côte-des-Neiges - Notre-Dame-de-Grâce',
                  'Lachine',
                  'Saint-Laurent',
                  'Ahuntsic - Cartierville',
                  'Sud-Ouest',
                  'Anjou',
                  'Montréal-Nord',
                  'Verdun',
                  'Pierrefonds - Roxboro']
cache = Cache(2e9)
cache.register()

In [4]:
init_centroids = park_spot_data.sample(frac=0.001).head(len(list_of_borough),npartitions=-1)

In [5]:
centroids = init_centroids.reset_index(drop=True).rename(columns={'sNoPlace':'name'})

In [6]:
park_spot_data.head()

,sNoPlace,nLongitude,nLatitude
0,A007,-73.581856,45.489593
1,A008,-73.581856,45.489593
2,A009,-73.581727,45.489532
3,A010,-73.581727,45.489532
4,A011,-73.581597,45.489471


In [6]:
park_spot_data['centroids']=''
park_spot_data.head()

,sNoPlace,nLongitude,nLatitude,centroids
0,A007,-73.581856,45.489593,
1,A008,-73.581856,45.489593,
2,A009,-73.581727,45.489532,
3,A010,-73.581727,45.489532,
4,A011,-73.581597,45.489471,


In [7]:
centroids.head()

,name,nLongitude,nLatitude
0,A739,-73.579919,45.489349
1,G291,-73.567038,45.510416
2,C849,-73.576579,45.497713
3,H780,-73.561963,45.511533
4,MA683,-73.541533,45.549377


In [32]:
from functools import reduce

def assignCentroid(row, centroids):
    c = reduce((lambda a,b: a if a[1]<b[1] else b),[(centroids.name,(row.nLongitude-centroids.nLongitude)**2 + (row.nLatitude-centroids.nLatitude)**2) for centroids in centroids.itertuples()]) 
    return  c[0]
park_spot_data['new_centroids'] = park_spot_data.apply(assignCentroid, axis=1, args=(centroids,), meta=('centroid', 'f8'))
#park_spot_data.head()

In [33]:
len(park_spot_data[park_spot_data['centroids']!=park_spot_data['new_centroids']])

533

In [34]:
centroids = park_spot_data[['nLongitude','nLatitude','new_centroids']].groupby('new_centroids').mean().reset_index().rename(columns={'new_centroids':'name'})

#centroids.head()

In [31]:
park_spot_data['centroids']=park_spot_data['new_centroids']

In [12]:
groups = park_spot_data[['nLongitude','nLatitude','new_centroids']].groupby('new_centroids')
groups

In [109]:
for name, group in groups:
    plt.plot(group["nLongitude"], group["nLatitude"], marker="o", linestyle="", label=name)
plt.show()

KeyError: 'Column not found: 0'